# Sketch the prototype to convert a graphml node description file to BIDS TSV format

## Define working directory

In [1]:
work_dir = "/Users/sebastientourbier/Documents/Postdoc/"

## Define a Nipype interface

In [2]:
from nipype.interfaces.base import (
    BaseInterfaceInputSpec,
    BaseInterface,
    TraitedSpec,
    File
)

class CreateBIDSStandardParcellationLabelIndexMappingFile_InputSpec(BaseInterfaceInputSpec):
    """Specify the inputs of the :obj:`~cmtklib.bids.utils.CreateBIDSStandardParcellationLabelIndexMappingFile`."""
    roi_graphml = File(
        mandatory=True,
        exists=True,
        desc="Path to graphml file that describes graph nodes for a given parcellation"
    )
    roi_colorlut = File(
        mandatory=True,
        exists=True,
        desc="Path to FreesurferColorLUT.txt file that describes the RGB color of the "
             "graph nodes for a given parcellation"
    )
    
class CreateBIDSStandardParcellationLabelIndexMappingFile_OutputSpec(BaseInterfaceInputSpec):
    """Specify the output of the :obj:`~cmtklib.bids.utils.CreateBIDSStandardParcellationLabelIndexMappingFile`."""
    roi_bids_tsv = File(
        exists=True,
        desc="Output BIDS standard generic label-index mapping file that "
             "describes parcellation nodes"
    )
    
class CreateBIDSStandardParcellationLabelIndexMappingFile(BaseInterface):
    """Creates the BIDS standard generic label-index mapping file that describes parcellation nodes"""
    
    input_spec = CreateBIDSStandardParcellationLabelIndexMappingFile_InputSpec
    output_spec = CreateBIDSStandardParcellationLabelIndexMappingFile_OutputSpec
    
    def _run_interface(self, runtime):
        import numpy as np
        import re
        import csv
        import networkx as nx

        # Extract code mapping from parcellation freesurfer color lookup table
        with open(self.inputs.roi_colorlut, 'r') as f:
            lut_content = f.readlines()

        # Process line by line
        rois_rgb = np.empty((0, 4), dtype=np.int64)
        pattern = re.compile(r'\d{1,5}[ ]+[a-zA-Z-_0-9*.]+[ ]+\d{1,3}[ ]+\d{1,3}[ ]+\d{1,3}[ ]+\d{1,3}')
        for line in lut_content:
            if pattern.match(line):
                s = line.rstrip().split(' ')
                s = list(filter(None, s))
                rois_rgb = np.append(rois_rgb, np.array([[int(s[0]), int(s[2]), int(s[3]), int(s[4])]]), axis=0)
                
        # Read the graphml node description file
        nodes_g = nx.readwrite.graphml.read_graphml(work_dir + "sub-01_atlas-L2018_res-scale2_dseg.graphml")
        nodes = nodes_g.nodes(data=True)
        del nodes_g
        
        # Create a dictionary conformed to BIDS with index, name, color, and mapping columns
        output_bids_node_description = []
        for node in nodes:
            in_node_description = node[1]
            out_node_description = {}
            out_node_description['index'] = in_node_description['dn_multiscaleID']
            out_node_description['name'] = in_node_description['dn_name'].lower()

            # Convert RGB color to hexadecimal 
            r, g, b = (rois_rgb[rois_rgb[:,0]==out_node_description['index']][:,1],
                       rois_rgb[rois_rgb[:,0]==out_node_description['index']][:,2],
                       rois_rgb[rois_rgb[:,0]==out_node_description['index']][:,3])
            out_node_description['color'] = '#%02x%02x%02x' % (r.squeeze(), g.squeeze(), b.squeeze())
            
            if 'brainstem' in in_node_description['dn_name']:
                out_node_description['mapping'] = 10
            else:
                if 'subcortical' in in_node_description['dn_region']:
                    out_node_description['mapping'] = 9
                elif 'cortical' in in_node_description['dn_region']:
                    out_node_description['mapping'] = 8

            output_bids_node_description.append(out_node_description)
            
        # Write output TSV file
        keys = ['index', 'name', 'color', 'mapping']
        output_tsv_filename = self._gen_output_filename(self.inputs.roi_graphml)
        with open(output_tsv_filename, 'w') as output_tsv_file:
            dict_writer = csv.DictWriter(output_tsv_file, keys, delimiter='\t')
            dict_writer.writeheader()
            dict_writer.writerows(output_bids_node_description)

        return runtime
    
    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs["roi_bids_tsv"] = self._gen_output_filename(self.inputs.roi_graphml)
        
    def _gen_output_filename(self, input_file):
        from pathlib import Path
        fpath = Path(input_file)
        return str(fpath.stem) + '.tsv'


## Test the interface

In [3]:
create_bids_code_description = CreateBIDSStandardParcellationLabelIndexMappingFile()
create_bids_code_description.inputs.roi_graphml = work_dir + "sub-01_atlas-L2018_res-scale2_dseg.graphml"
create_bids_code_description.inputs.roi_colorlut = work_dir + "sub-01_atlas-L2018_res-scale2_FreeSurferColorLUT.txt"
create_bids_code_description.run()